In [1]:
import os
import warnings
from dotenv import load_dotenv

os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
warnings.filterwarnings("ignore")

load_dotenv()

True

### Doc Loader

In [5]:
from langchain_community.document_loaders import PyMuPDFLoader

file_path = "./data/gym/1_Analysis_of_Actual_Fitness_Supplement.pdf"
loader = PyMuPDFLoader(file_path)

docs = loader.load()

In [ ]:
doc = docs[0]
# print(doc.page_content)

In [14]:
import os

pdfs = []
for root, dirs, files in os.walk('data'):
    # print(root, dirs, files)
    for file in files:
        if file.endswith('.pdf'):
            pdfs.append(os.path.join(root, file))

In [16]:
docs = []
for pdf in pdfs:
    loader = PyMuPDFLoader(pdf)
    pages = loader.load()

    docs.extend(pages)


### Document Chunking

In [18]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
)

chunks = text_splitter.split_documents(docs)

In [20]:
len(docs), len(chunks)

(32, 152)

In [22]:
print(docs[0].page_content)

International  Journal  of
Environmental Research
and Public Health
Review
Dietary Supplements—For Whom? The Current State of
Knowledge about the Health Effects of Selected
Supplement Use
Regina Ewa Wierzejska


Citation: Wierzejska, R.E. Dietary
Supplements—For Whom? The
Current State of Knowledge about the
Health Effects of Selected Supplement
Use. Int. J. Environ. Res. Public Health
2021, 18, 8897. https://doi.org/
10.3390/ijerph18178897
Academic Editor: Paul B. Tchounwou
Received: 15 July 2021
Accepted: 21 August 2021
Published: 24 August 2021
Publisher’s Note: MDPI stays neutral
with regard to jurisdictional claims in
published maps and institutional afﬁl-
iations.
Copyright: © 2021 by the author.
Licensee MDPI, Basel, Switzerland.
This article is an open access article
distributed
under
the
terms
and
conditions of the Creative Commons
Attribution (CC BY) license (https://
creativecommons.org/licenses/by/
4.0/).
Department of Nutrition and Nutritional Value of Food, 

In [23]:
print(chunks[0].page_content)

International  Journal  of
Environmental Research
and Public Health
Review
Dietary Supplements—For Whom? The Current State of
Knowledge about the Health Effects of Selected
Supplement Use
Regina Ewa Wierzejska


Citation: Wierzejska, R.E. Dietary
Supplements—For Whom? The
Current State of Knowledge about the
Health Effects of Selected Supplement
Use. Int. J. Environ. Res. Public Health
2021, 18, 8897. https://doi.org/
10.3390/ijerph18178897
Academic Editor: Paul B. Tchounwou
Received: 15 July 2021
Accepted: 21 August 2021
Published: 24 August 2021
Publisher’s Note: MDPI stays neutral
with regard to jurisdictional claims in
published maps and institutional afﬁl-
iations.
Copyright: © 2021 by the author.
Licensee MDPI, Basel, Switzerland.
This article is an open access article
distributed
under
the
terms
and
conditions of the Creative Commons
Attribution (CC BY) license (https://
creativecommons.org/licenses/by/
4.0/).
